# Neuromap Project Neurohackademy 2019

In [6]:
# install libraries
import pandas as pd
import folium
import geopy

In [7]:
# sets the defaults for viewing the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
#load in data 
df = pd.read_csv('NeuroMap-38responses.csv', encoding='latin-1')

#view data
df.head()

#Get the column with hometown location
locs = df.iloc[:,3]

#Print locs - check for typos, etc
#locs

In [9]:
# #clean data
# # rename column questions to shorter
df.columns = ['timestamp','consent','name','birthplace','birth_year', 'gender',
              'undergrad_deg','undergrad_loc',  'undergrad_inst','undergrad_research',
              'ra_qual','ra_lm_loc', 'ra_lm_inst', 'ra_lm_research',
              'masters_qual','masters_loc', 'masters_inst', 'masters_research', 
              'phd_qual','phd_loc', 'phd_inst', 'phd_research',
              'post_doc_qual', 'postdoc_loc' ,'postdoc_inst','postdoc_research', 
            'faculty_qual', 'faculty_loc', 'faculty_inst', 'faculty_research',
              'google_scholar']

df.head()

,timestamp,consent,name,birthplace,birth_year,gender,undergrad_deg,undergrad_loc,undergrad_inst,undergrad_research,ra_qual,ra_lm_loc,ra_lm_inst,ra_lm_research,masters_qual,masters_loc,masters_inst,masters_research,phd_qual,phd_loc,phd_inst,phd_research,post_doc_qual,postdoc_loc,postdoc_inst,postdoc_research,faculty_qual,faculty_loc,faculty_inst,faculty_research,google_scholar
0,2019/08/06 12:34:12 pm GMT-6,Yes,Antonija Kolobaric,"Split, Croatia",1994.0,Female,Experimental Psychology,"Providence, RI, USA",Brown University,"Theory of mind, suicide, development, adolesce...",Yes,"New Haven, CT, USA",Yale University,"early course psychosis, thalamus, mental illne...",No,NaN,NaN,NaN,Yes,"Pittsburgh, PA, USA",University of Pittsburgh,"development, psychosis, fMRI, thalamus",No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
1,2019/08/06 12:37:29 pm GMT-6,Yes,Giacomo,"Verona, Veneto, Italy",1988.0,Male,Cognitive Psychology,"Rovereto, Trentino, Italy",Unversity of Trento,"healthy, behavioral tracking with IR markers",No,NaN,NaN,NaN,Yes,"Rovereto, Trentino, Italy",University of Trento,"healthy, eyetracking on pointlight displays",Yes,"Rovereto, Trentino, Italy",University of Trento,"healthy, fMRI, movement decoding",Yes,"London, Ontario, Canada",Western University,"healthy, fMRI, skill learning, motor represent...",No,NaN,NaN,NaN,https://scholar.google.com/citations?user=IYbg...
2,2019/08/06 12:38:08 pm GMT-6,Yes,Hannah Coyle,"Northampton, United Kingdom",1991.0,Female,Psychology,"Wollongong, NSW, Australia","University of Wollongong, Australia","cannabis use, EEG, working memory",Yes,"Melbourne, Victoria, Australia",Monash University,"depression, TMS, DLPFC",No,NaN,NaN,NaN,Yes,"Melbourne, Victoria, Australia",Monash University,"mild traumatic brain injury, TMS-EEG, neuropsy...",No,NaN,NaN,NaN,No,NaN,NaN,NaN,https://scholar.google.com/citations?view_op=l...
3,2019/08/06 12:39:18 pm GMT-6,Yes,JKL,Hong Kong,1986.0,Male,Psychology,Hong Kong,University of Hong Kong,"sexual health, clinical, survey",Yes,"Birmingham, UK",University of Birmingham,"stroke, cognitive assessment battery",Yes,"Birmingham, UK",University of Birmingham,"developmental disabilities, eating, cognitive ...",Yes,"Birmingham, UK",University of Birmingham,"temporal, parietal areas; lesion analysis, fmr...",Yes,"Reading, UK",University of Reading,"reward network, striatum, medial frontal areas...",No,NaN,NaN,NaN,NaN
4,2019/08/06 12:51:26 pm GMT-6,Yes,Liz B,"Chicago, IL, USA",1992.0,Female,Communications & Psychology,"Ann Arbor, MI, USA",University of Michigan,"Communication neuroscience, social influence, ...",Yes,"Philadelphia, PA, USA",University of Pennsylvania,"communication neuroscience, smokers, fMRI, tD...",No,NaN,NaN,NaN,Yes,"Philadelphia, PA, USA",Temple University,"adolescent decision-making, fMRI, reward proce...",No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN


In [10]:
#
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="neuromap")

#Loop through rows to get the longitude and latitude of the hometowns
lat=[]
long=[]
for location in df.loc[:,"birthplace"]:
    location_1 = geolocator.geocode(location, timeout=10)
    #print((location_1.latitude, location_1.longitude))
    
    lat.append(location_1.latitude)
    long.append(location_1.longitude)

#Saving lat and long in separate columns in the dataframe    
df['HometownLatitude']=lat
df['HometownLatitude'].astype('float')

df['HometownLongitude']=long
df['HometownLongitude'].astype('float')

0      16.441423
1      10.992412
2      -0.902493
3     114.162813
4     -87.624421
5     -97.337545
6     -58.437076
7      56.246723
8      72.835335
9     -84.512460
10    -86.875287
11    -77.152757
12    121.563680
13     80.618510
14    -73.949582
15    129.075236
16   -116.964663
17     34.790579
18      4.481109
19    -92.561787
20    -98.493387
21   -100.445882
22   -118.319612
23    -43.209373
24     36.817245
25     34.780527
26    -78.639099
27     82.923451
28    -94.626497
29   -122.271356
30     51.401378
31    -74.006015
32    -70.254860
33    -81.229153
34     35.225079
35    -74.203757
36    -74.659296
37     11.125760
Name: HometownLongitude, dtype: float64

In [11]:
#Try to plot the locations on the map!
m = folium.Map()

#(location=[location_1.latitude, location_1.longitude])

#Loop through locations and add the markers on the map
for location in range(len(locs)): 
    folium.Marker([lat[location], long[location]], popup=locs[location]).add_to(m)

m


In [ ]:
def generateBaseMap(default_location=[-34.4243941, 150.89385], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
base_map

In [ ]:
m= folium.Map(default_location=[-34.4243941, 150.89385])

In [ ]:
m = folium.Map(
    location=[-34.4243941, 150.89385],
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[-34.4243941, 150.89385],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[-34.4243941, 150.89385],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


m